In [1]:
import numpy as np
import os
home_path = os.path.expanduser("~") + '/'
from utils.load_data import *
from utils.info_params import *
from sklearn.svm import SVC

OpenCV is built with OpenMP support. This usually results in poor performance. For details, see https://github.com/tensorpack/benchmarks/blob/master/ImageNet/benchmark-opencv-resize.py


In [2]:
def get_data(hps):
    dfX, df_next_deltaClose = load_data_seq(hps)

    segment, next_segment, target_one_hot = segment_seq(dfX, df_next_deltaClose, hps)

    train_segment, test_segment, _, _, train_target_one_hot, test_target_one_hot = \
        train_test_split(segment, next_segment, target_one_hot, hps)

    return train_segment, test_segment, train_target_one_hot, test_target_one_hot

In [3]:
hps = get_default_hparams()
hps.data_file_name = home_path + "data/ccxt/extra/BTC_USDT_binance_1h.csv"

hps.attributes_normalize_mean = ['Low','Close','Volume','N_buy','N_sell','Spread_Open_Close','Spread_High_Low','MA_Close_240']

hps.is_concat = False
hps.Tau = 1
hps.T = 20
hps.C = 2
hps.D = (1+hps.is_concat) * len(hps.attributes_normalize_mean)
hps.is_differencing = True
hps.lag_time = 1
hps.N_train_seq = 10000 + hps.T - 1
hps.normalize_data = 'z_score'
hps.normalize_data_idx = True

In [4]:
X_train, X_test, train_target_one_hot, test_target_one_hot = get_data(hps)
X_train = np.reshape(X_train, newshape=[-1, hps.T * hps.D])
X_test = np.reshape(X_test, newshape=[-1, hps.T * hps.D])

y_train = np.argmax(train_target_one_hot, axis=-1)
y_train = np.reshape(y_train, newshape=[-1, hps.Tau])

y_test = np.argmax(test_target_one_hot, axis=-1)
y_test = np.reshape(y_test, newshape=[-1, hps.Tau])

Normalize: Z score


In [5]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/home/nam/miniconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_train_hat = clf.predict(X_train)
cm = confusion_matrix(y_train, y_train_hat)
rp = classification_report(y_train, y_train_hat)
score = accuracy_score(y_train, y_train_hat)

print(cm)
print(rp)
print(score)

[[1617 3223]
 [1148 4012]]
              precision    recall  f1-score   support

           0       0.58      0.33      0.43      4840
           1       0.55      0.78      0.65      5160

   micro avg       0.56      0.56      0.56     10000
   macro avg       0.57      0.56      0.54     10000
weighted avg       0.57      0.56      0.54     10000

0.5629


In [7]:
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
rp = classification_report(y_test, y_pred)
score = accuracy_score(y_test, y_pred)

print(cm)
print(rp)
print(score)

[[1005 2596]
 [1016 2864]]
              precision    recall  f1-score   support

           0       0.50      0.28      0.36      3601
           1       0.52      0.74      0.61      3880

   micro avg       0.52      0.52      0.52      7481
   macro avg       0.51      0.51      0.49      7481
weighted avg       0.51      0.52      0.49      7481

0.5171768480149712


In [1]:
# #Saving model
# from sklearn.externals import joblib
# joblib.dump(clf, './checkpoint/svm_sklearn-origin_data.joblib') 

In [ ]:
from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np

print(__doc__)

# Number of random trials
NUM_TRIALS = 30

# Set up possible values of parameters to optimize over
p_grid = {"C": [1, 10, 100],
          "gamma": [.01, .1]}

# We will use a Support Vector Classifier with "rbf" kernel
svm = SVC(kernel="rbf")

# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv,
                       iid=False)
    clf.fit(X, y)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X, y=y, cv=outer_cv)
    nested_scores[i] = nested_score.mean()

score_difference = non_nested_scores - nested_scores

print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on Iris Dataset",
          x=.5, y=1.1, fontsize="15")

# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()